In [ ]:
import plotly

import plotly.express as px
import pandas as pd
import numpy as np
import plotly.io as pio

# enables the %%R magic to run python and R in different cells
%load_ext rpy2.ipython

In [ ]:
def get_cofact_names(df, rename_dict, nestedlist=False):
    '''
    for a given dataframe, turns the column containing the Uniprot cofactor field 
    into a list with cofactor names
    '''
    cofact_name = []
    for i, na in zip(df['Cofactor'], df['Cofactor'].isna()):
        if na:
            cofact_name.append(i)
        if not na:
            names = [j.split('=')[-1] for j in i.split(';') if 'COFACTOR' in j]
            #if defined in dictionary, rename cofactor
            for ind, name in enumerate(names):
                #classify the really long ones as 'other'
                if len(name) > 35:
                    names[ind] = 'Other'
                if name in rename_dict:
                    names[ind] = rename_dict[name]
            if nestedlist:
                cofact_name.append(names)
            else:
                cofact_name+=names
    return cofact_name


def integrate_cosubstrate(df, df_cosub, cosub_name):
    '''
    given a dataframe of enzymes (with uniprot Entry as a column)
    and a dataframe of enzymes with a certain cosubstrate, 
    integrates the cosubstrate into the cofactor column of the first dataframe
    '''
    df_out = df.copy()
    cosub_notna = df.Entry.isin(df_cosub.Entry) & df.Cofactor.notna()
    cosub_na = df.Entry.isin(df_cosub.Entry) & df.Cofactor.isna()

    df_out.loc[cosub_notna, 'Cofactor'] = df[cosub_notna].Cofactor.apply(lambda x:x+[cosub_name])
    df_out.loc[cosub_na, 'Cofactor'] = df[cosub_na].Cofactor.apply(lambda x:[cosub_name])
    return df_out


#dictionary of cofactor names that should be renamed in order to group or shorten for clarity
rename_cofactors={"pyridoxal 5'-phosphate":"PLP", 
                  "S-adenosyl-L-methionine":"SAM",
                  "thiamine diphosphate":"TPP",
                  "a divalent metal cation":"Metal(2+)",
                  "Cu cation":"Cu(2+)",
                  "Fe cation":"Fe(2+)",
                  "Ni cation":"Ni(2+)",
                  "siroheme":"heme",
                  "ferriheme a":"heme",
                  "heme b":"heme",
                  "heme c":"heme",
                  "heme d cis-diol":"heme",
                  "Fe(II)-heme o":"heme",
                  "Heme A3.":"heme",
                  "Heme A3. {ECO:0000250}":"heme",
                  "NADPH":"NADP(+)",
                  "FMNH2":"FMN",
                  "prenyl-FMN":"FMN",
                  "iron-sulfur cluster":"Fe-S cluster",
                  "[4Fe-4S] cluster":"Fe-S cluster",
                  "[2Fe-2S] cluster":"Fe-S cluster",
                  "[3Fe-4S] cluster":"Fe-S cluster",
                  "[8Fe-7S] cluster":"Fe-S cluster",
                  "[7Fe-Mo-9S-C-homocitryl] cluster":"Fe-S cluster",
                  "[Ni-4Fe-4S] cluster":"Ni-Fe-S cluster",
                  "[Ni-4Fe-5S] cluster":"Ni-Fe-S cluster",
                  "[Ni-Fe-S] cluster":"Ni-Fe-S cluster",
                  "hybrid [4Fe-2O-2S] cluster":"Fe-S cluster",
                  "Mo-molybdopterin":"MPT",
                  "Mo-bis(molybdopterin guanine dinucleotide)":"MPT",
                  "Mo-molybdopterin cytosine dinucleotide":"MPT",
                  "W-bis(molybdopterin guanine dinucleotide)":"MPT",
                  "Has a very strong preference for NAD(+) over NADP(+). {ECO:0000250|UniProtKB:P41793}":"NAD(+)",
                 }

In [ ]:
#Load in all data to create processed dataframe
#============================================================================================================
#load in uniprot data for all (e. coli) proteins, including Entry name, EC number,and info on their cofactor
#for all prot: (reviewed:true)
all_prot = pd.read_csv('uniprot-all-GO-2023.02.15-13.12.03.40.tsv', sep='\t')

all_cofact = get_cofact_names(all_prot, rename_cofactors, nestedlist=True)
all_prot['Cofactor'] = all_cofact

#for coenzymes uniprot sometimes considers them cosubstrates and thefore does not include them as cofactor.
#the list for these can be downloaded sperately and integrated into the data

#for NAD/NADH: ((chebi:16908) OR (chebi:13389)) AND (reviewed:true)
all_nad = pd.read_csv('uniprot-NAD-all-2023.02.13-09.52.03.54.tsv', sep='\t')
#for NADP/NADPH: ((chebi:44409) OR (chebi:16474)) AND (reviewed:true)
all_nadp = pd.read_csv('uniprot-NADP-all-2023.02.13-09.53.00.12.tsv', sep='\t')
#for SAM: (chebi:67040) AND (reviewed:true)
all_sam = pd.read_csv('uniprot-SAM-all-2023.02.13-09.54.04.42.tsv', sep='\t')
#for ATP: (chebi:15422) AND (reviewed:true)
all_atp = pd.read_csv('uniprot-ATP-all-2023.02.13-09.54.55.17.tsv', sep='\t') 
#for GTP: (chebi:15996) AND (reviewed:true)
all_gtp = pd.read_csv('uniprot-GTP-all-2023.02.13-09.56.04.71.tsv', sep='\t')

#integrate all the other cofactors into the list
all_prot = integrate_cosubstrate(all_prot, all_nad, 'NAD(+)')
all_prot = integrate_cosubstrate(all_prot, all_nadp, 'NADP(+)')
all_prot = integrate_cosubstrate(all_prot, all_sam, 'SAM')
all_prot = integrate_cosubstrate(all_prot, all_atp, 'ATP')
all_prot = integrate_cosubstrate(all_prot, all_gtp, 'GTP')

#use gene ontology to make column on DNA/RNA binding
gene_o = all_prot['Gene Ontology (molecular function)'].copy()
gene_o.loc[gene_o.isna()] = 'no'
dna_binding = gene_o.apply(lambda x:(('DNA binding' in x)))
rna_binding = gene_o.apply(lambda x:(('RNA binding' in x)))
all_prot['DNA'] = dna_binding
all_prot['RNA'] = rna_binding

#drop the gene ontology column
all_prot = all_prot.drop('Gene Ontology (molecular function)', axis=1)

#create column with enzyme/not enzyme, and cofactor present/not present
all_prot['Enzyme'] = all_prot['EC number'].notna()
all_prot['Cofactor present'] = all_prot['Cofactor'].notna()

#output processed dataframe
#============================================================================================================

all_prot_out = all_prot.copy()
all_prot_out.loc[all_prot.Cofactor.notna(), 'Cofactor'] = all_prot_out[all_prot.Cofactor.notna()].Cofactor.apply(lambda x:'  '.join(x))
all_prot_out.loc[all_prot.Cofactor.isna(), 'Cofactor'] = all_prot_out[all_prot.Cofactor.isna()].Cofactor.apply(lambda x:x)

all_prot_out.to_csv('uniprot-all-prot-processed.tsv', sep="\t")

In [ ]:
#load in processed dataframe
#============================================================================================================
all_prot = pd.read_csv('uniprot-all-prot-processed.tsv', sep='\t')
all_prot.loc[all_prot.Cofactor.notna(), 'Cofactor'] = all_prot[all_prot.Cofactor.notna()].Cofactor.apply(lambda x:str(x).split('  '))


In [ ]:
#Create a treemap
#============================================================================================================

list_inorganic = ['Fe-S cluster', 'Ni-Fe-S cluster', 'Mg(2+)', 'Zn(2+)', 'Mn(2+)', 'Fe(2+)', 'Metal(2+)', 
                'Ca(2+)', 'K(+)', 'Cu(2+)', 'Ni(2+)', 'Co(2+)', 'Co(3+)',
                'Cu(+)', 'Fe(3+)', 'a metal cation', 'a monovalent cation', 'vanadium cation',
                  'phosphate', 'chloride', 'NH4(+)', 'Na(+)'
               ]

#make into big list of cofactors
all_cofact= []
for i in all_prot.Cofactor:
    if type(i) == list:
        all_cofact+=i
        
#seperate into organic and inorganic
org_inorg = []
for i in all_cofact:
    if i in list_inorganic:
        org_inorg.append('Inorganic')
    else:
        org_inorg.append('Organic')
        
#restructure list of cofactors to be compatible with treemap
all_cofact = pd.DataFrame({'Class':org_inorg,'Cofactor':all_cofact})
all_cofact = all_cofact.groupby(all_cofact.columns.tolist(), dropna=False, as_index=False).size()


#make the version with a magenta colorscale
fig = px.treemap(all_cofact, path=['Class', 'Cofactor'], values='size', color='size',
               color_continuous_scale='Magenta', 
               range_color=[0, 30000]) 
fig.data[0].textinfo = 'label+text+value'

fig.show()
fig.update_layout(font=dict(size=30))

pio.write_image(fig, 'treemap-pink.png',scale=6, width=1180, height=1080)

#make the version with a magenta colorscale
fig = px.treemap(all_cofact, path=['Class', 'Cofactor'], values='size', color='size',
               color_continuous_scale='Emrld', 
               range_color=[0, 30000]) 
fig.data[0].textinfo = 'label+text+value'

fig.show()
fig.update_layout(font=dict(size=30))

pio.write_image(fig, 'treemap-green.png',scale=6, width=1180, height=1080)

#the pink and green versions were combined using photoshop

In [ ]:
#Create a sunburst plot
#============================================================================================================
df = all_prot.copy()
#check if ther is either DNA or RNA present
nucl_binding = df['DNA'] | df['RNA']
#column for whether or not entry is an enzyme depending on if the EC number is NaN, 
# column for whether cofactor is present based on if Cofactor is NaN
Enzyme = df['EC number'].isna().map({True: 'Not Enzyme', False: 'Enzyme'})
Cofactor_present = df['Cofactor'].isna().map({True: 'No Cofactor', False: ' Cofactor'}) 
Cofactor_present.loc[dna_binding] = Cofactor_present.loc[dna_binding].map({'No Cofactor':' Nucl. acid', ' Cofactor':' Cofactor+Nucl'}) 
#go over each cofactor entry. skip if NaN, shorten/rename if it has an entry
cofact_name = []
cofact_nr = []
for i, na in zip(df['Cofactor'], df['Cofactor'].isna()):
    if na:
        cofact_nr.append(i)
    if not na:
        #names = [j.split('=')[-1] for j in i.split(';') if 'COFACTOR' in j]    
        cofact_nr.append(str(len(i)))
#prepare dataframe for sunburst plot and group by all columns to get count of each occurence
df_sunburst = pd.DataFrame({'Enzyme':Enzyme, 'Cofactor present':Cofactor_present, 
                            'Cofactor count':cofact_nr})
all_sunburst = df_sunburst.groupby(df_sunburst.columns.tolist(), dropna=False, as_index=False).size()

fig = px.sunburst(all_sunburst, path=['Enzyme', 'Cofactor present', 'Cofactor count'], values='size')
#makes it sort alphabetically instead of on value size. can be manipulated by adding spaces
fig.update_traces(sort=False) 
fig.show()
fig.update_layout(font=dict(size=18))
pio.write_image(fig, 'sunburst.png',scale=6, width=1080, height=1080)

In [ ]:
#Create a pie/doughnut chart using R
#============================================================================================================
%%R
library(tidyverse)
library(ggpattern)

rows <- apply(expand.grid(LETTERS, LETTERS), 1, function(x) paste0(x, collapse=''))

df_real <- read_tsv('uniprot-all-prot-processed.tsv', show_col_types = F) %>% as.data.frame()
df_real <- df_real[,c(1,2,3,4,5,6,8,12,11,7,9,10)]
# df <- df_real[df_real$`Organism (ID)` == 83333,9:12]
df <- df_real[,9:12]
colnames(df) <- c('enz', 'cof', 'dna', 'rna')

df$cof <- sapply(df_real[,'Cofactor'], function(x) {
  out <- str_split(x, "  ")[[1]]
  length(out[out != "" & !is.na(out)])
})

df <- df[order(df$cof),]
df <- apply(df, 2, as.integer)
df <- as.data.frame(df)
df$na <- df$dna
df[df$rna == 1, 'na'] <- 1


dfc <- count(df, enz, cof, na)
dfc <- dfc[order(dfc$enz, dfc$cof),]
dfc$na <- c('no', 'yes')[dfc$na + 1]

dfc$xmax <- cumsum(dfc$n)
dfc$xmin <- c(0,dfc$xmax[-nrow(dfc)])

sepspace <- max(dfc$xmax * 0.004)

dfc[dfc$enz != c(1,dfc$enz[-length(dfc$enz)]), 'xmin'] <- dfc[dfc$enz != c(1,dfc$enz[-length(dfc$enz)]), 'xmin'] + sepspace
dfc$coff <- factor(dfc$cof)

pie <- data.frame(enz = c(0,1), 
                  n = c(sum(dfc[dfc$enz == 0, 'n']),
                        sum(dfc[dfc$enz == 1, 'n'])))
pie$xmax <- cumsum(pie$n)
pie$xmin <- c(0,pie$xmax[-nrow(pie)])

pie$xmax <- pie$xmax / max(pie$xmax) * max(dfc$xmax)
pie$xmin <- c(0,pie$xmax[-nrow(pie)])

pie$xmin <- pie$xmin + sepspace
pie$enzf <- c('noenz', 'enz')[pie$enz + 1]

rot <- pie[pie$enz == max(pie$enz), 'n'] / sum(pie$n) * 360 /2 

ggplot(dfc) +
  # geom_hline(data = data.frame(l = c(0,1,2,3,4)), aes(yintercept = l), size=0.1, color = 'grey70') +
  # geom_line(data = data.frame(l = rep(c(0,1,2,3,4), each=2), x= rep(c(0,max(dfc$xmax)), 5)), 
  #           aes(x = x,y = l, group=l), size=0.1, color = 'grey33', alpha = 0.25) +
  # 
  geom_line(data = data.frame(l = rep(0:5, each=2), x= rep(c(0,sepspace *0.8), 6)), 
            aes(x = x,y = l, group=l), size=0.3, color = 'black') +
  
  coord_polar(start = rot * pi / 180) +
  scale_y_continuous(expand=c(0,0),limits = c(-8,5), breaks=0:5) +
  scale_x_continuous(limits = c(0,max(dfc$xmax))) +
  theme_bw() +
  geom_rect(data=pie,aes(xmin=xmin,xmax=xmax,ymin=-8,ymax=-1.1, fill=enzf)) +
  geom_line(data = data.frame(l = rep(c(0,1), each=2), x= rep(pie$xmax, each=2), y=rep(c(-8,4),2)),
            aes(x = x,y = y, group=l), size=1.8, color = 'white') +
  geom_rect_pattern(aes(xmin = xmin,
                        xmax = xmax,
                        ymin = -1,
                        ymax = cof,
                        fill = coff,
                        pattern_angle = (xmin + xmax )/2 / max(xmax) *180 ,
                        pattern_fill = na),
                    color = 'grey95',
                    pattern_colour = 'transparent',
                    pattern_density = 0.2,
                    pattern_spacing = 0.005,
                    linewidth = 0.3) +
  scale_pattern_fill_manual(name = "Nucleic acid-binding", values = c(no = 'transparent', yes = 'grey15')) +
  # geom_text(aes(x = (xmin + xmax) / 2, y = (cof -1) /2, label = signif(n / sum(n) * 100,2) %>% str_replace('^0$', ''),
  #               # angle = round(-90 + (-360 * (((xmin + xmax) / 2) / max(((xmin + xmax) / 2)))),0)
  # ), size = 2.5) +
  scale_fill_manual(values = c('enz'='#7412ff',
                               'noenz' ='#c0bdff',
                               '0' = 'grey95',
                               '1' ='#b4efff',
                               '2' ='#00c9ff',
                               '3' ='#00a2ff',
                               '4' ='blue',
                               '6' ='black')) +
  guides(pattern_fill = guide_legend(override.aes = list(fill = "grey95"))) +
  # scale_fill_gradientn(colours = c('white', '#56dbff', 'blue', '#b3ffe9','#b3ffe9', '#4affcb', '#00a587')) +
  # scale_fill_gradientn(colours = c('white', '#00c9ff', 'blue', '#b3ffe9','#b3ffe9', '#4affcb', '#00a587')) +
  theme_void() +
  theme(axis.text = element_blank(),
        axis.title = element_blank(),
        axis.ticks = element_blank(),
        panel.border = element_blank(),

        panel.grid.major.x = element_blank(),
        panel.grid.minor = element_blank()) +
  # annotate('text', x=sum(dfc$n) /4 - ((rot /360) * sum(dfc$n)), y=(c(0,1,2,3,4) - 0.45),label=(c(0,1,2,3,4)), size = 4, color = 'grey33')
  annotate('text', x=sepspace *2, y=(c(1,2,3,4,5)),label=(c(1,2,3,4,5)), size = 4, color = 'grey33')

